In [98]:
import pandas as pd
import os
import pickle

In [99]:
cache_file = '../cache/bitcoin/data.pickle'
dataset_path = '../datasets/original/Bitcoin Historical Data(2010-2022).csv'

data = None

if os.path.isfile(cache_file):
    with open(cache_file, 'rb') as f:
        data = pickle.load(f)
else:
    data = pd.read_csv(
        filepath_or_buffer=dataset_path,
    )

    data.rename(columns=str.lower, inplace=True)

    data['date'] = pd.to_datetime(data['date']).dt.date

    data = data.sort_values(by='date').reset_index(drop=True)

    close = data.pop('close')

    data['close'] = close

    with open(cache_file, 'wb') as f:
        pickle.dump(data, f)

data

,date,open,high,low,volume,close
0,2010-07-18,0.0,0.1,0.1,75,0.1
1,2010-07-19,0.1,0.1,0.1,574,0.1
2,2010-07-20,0.1,0.1,0.1,262,0.1
3,2010-07-21,0.1,0.1,0.1,575,0.1
4,2010-07-22,0.1,0.1,0.1,2160,0.1
...,...,...,...,...,...,...
4301,2022-04-27,38113.0,39447.0,37869.0,426830240,39243.0
4302,2022-04-28,39243.0,40382.0,38888.0,542391872,39748.0
4303,2022-04-29,39748.0,39919.0,38175.0,382904352,38581.0
4304,2022-04-30,38590.0,38776.0,37596.0,248220336,37650.0


In [100]:
data = data.dropna().reset_index(drop=True)

data.isna().sum()

date      0
open      0
high      0
low       0
volume    0
close     0
dtype: int64

In [101]:
dataset_path = '../datasets/consolidated/posts.csv'

data_posts = pd.read_csv(
    filepath_or_buffer=dataset_path,
    lineterminator='\n',
)

data_posts['date'] = pd.to_datetime(data_posts['date']).dt.date

data_posts

,date,sentiment
0,2019-05-26,0.231749
1,2019-05-27,0.147698


In [105]:
from datetime import timedelta

min_date = data_posts['date'].min()
max_date = data_posts['date'].max()

data = data[data['date'] >= min_date]
data = data[data['date'] <= max_date + timedelta(days=3)]

data = data.reset_index(drop=True)

data

,date,open,high,low,volume,close
0,2019-05-26,8029.6,8718.6,7866.5,872118,8630.2
1,2019-05-27,8628.9,8902.8,8607.3,907961,8760.1
2,2019-05-28,8759.4,8794.1,8568.8,763203,8716.3
3,2019-05-29,8715.6,8747.2,8435.3,15429494,8647.8
4,2019-05-30,8647.8,9045.9,8054.7,1041563,8271.9


In [104]:
data.to_csv('../datasets/consolidated/bitcoin.csv', index=False)